<a href="https://colab.research.google.com/github/ThreeMachineExpression/contrapuntalPoetry/blob/main/PhoneticSimilarity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


Parts of this notebook are based on Max Woolf's aitextgen notebook, as updated by Allison Parrish.

### Setup

In [1]:
# Freeze versions of dependencies for now
!pip install tensorflow==1.15.0 keras==2.2.5 "h5py<3.0.0"
!pip3 install pytorch-lightning==0.7.6
!pip3 install transformers==2.9.1
!pip3 install fire==0.3.0

!pip install -q aitextgen==0.2.3

from aitextgen import aitextgen
from aitextgen.colab import mount_gdrive, copy_file_from_gdrive

mount_gdrive()

!pip install annoy
!pip install pronouncing
!pip install pincelate
import pronouncing
import pincelate
pin = pincelate.Pincelate()

from collections import Counter

import random
import textwrap
import numpy as np

import logging
logging.basicConfig(
        format="%(asctime)s — %(levelname)s — %(name)s — %(message)s",
        datefmt="%m/%d/%Y %H:%M:%S",
        level=logging.INFO
    )

kPhoneticSimilarityVectorsRepo = "https://github.com/aparrish/phonetic-similarity-vectors"

!git clone {kPhoneticSimilarityVectorsRepo}

!python -m spacy download en_core_web_md

%cd phonetic-similarity-vectors
from featurephone import phone_feature_map as pfm
%cd ..

pfm['STOP'] = ()

vowels = ['AO','AA','IY','UW','EH','IH','UH','AH','AE','EY','AY','OW','AW','OY',
          'ER','STOP']

# When deciding how close two syllables are, these constants indicate how much
# weight to give vowel features, stress, and consonant features
kDistanceWeightVowels = 4.0
kDistanceWeightStress = 5.0
kDistanceWeightConsonants = 1.0



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).



Using TensorFlow backend.



Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Cloning into 'phonetic-similarity-vectors'...
remote: Enumerating objects: 32, done.
remote: Total 32 (delta 0), reused 0 (delta 0), pack-reused 32
Unpacking objects: 100% (32/32), done.
     |████████████████████████████████| 96.4MB 1.2MB/s 
  Created wheel for en-core-web-md: filename=en_core_web_md-2.2.5-cp36-none-any.whl size=98051304 sha256=0729af81a74cba9ba95a67b8242f02d4390e8788dba7d598eb9016fa765fc537
  Stored in directory: /tmp/pip-ephem-wheel-cache-3lcml_vy/wheels/df/94/ad/f5cf59224cea6b5686ac4fd1ad19c8a07bc026e13c36502d81
Successfully built en-core-web-md
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_md')
/content/phonetic-similarity-vectors
/content


In [2]:
from_folder = "aitextgenDreamFineTuning"

for file in ["pytorch_model.bin", "config.json"]:
  if from_folder:
    copy_file_from_gdrive(file, from_folder)
  else:
    copy_file_from_gdrive(file)

In [3]:
ai = aitextgen(model="pytorch_model.bin", config="config.json", to_gpu=True)

INFO:aitextgen:Loading GPT-2 model from provided pytorch_model.bin.
INFO:aitextgen:Using the default GPT-2 Tokenizer.


# Generation

In [4]:
!nvidia-smi

Fri Jan 22 09:41:04 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P0    39W / 300W |   1931MiB / 16130MiB |     12%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [5]:
ai.generate()


You've wanted to create a story for a while only to have it leak out and into the void, where nothing real happens.
And then it's on the news instead of the people you know.
You try to turn it off for a while, but everyone keeps saying stupid things and it doesn't really work.
And then it all just goes back to pretending that everything's okay and everything's okay.
Like when the bright ball of light descends and lands in your throat and you can sing and everyone knows they're loved, the elderly women who are finally marrying feel in their bones that everything's okay and they really do love me.
I don't need anyone to love me anymore, I need someone to love me, and I NEEDED to hear that everyone was loved so much before.
I went to medical school so that I can provide proper care to the people I've hit over the head with 18th-century furniture.
Everything I need to know about this job interview is in the open field outside the hospital.
When I apply for this gig, there are rules agains

# Phonetic Similarity

In [315]:
# Strategy:
# Split text into syllable buckets
# In each bucket, include the vowel sound, stress, and all of the bordering phonemes
#  (so a consonant between syllables goes in both buckets)
# Measure the distance between two buckets as 
#   a * (manhattan distance of consonant features)
# + b * (manhattan distance of vowel features)
# + c * (stress distance)
# (with a, b, c tuned as desired to emphasize different similarities)
#
# Pauses get a bucket of their own.

def phones_for_word_fb(word):
  """Phones for word - either 1st entry in the CMU pronouncing dictionary
   or fallback to Pincelate sound-out if it's not in the dictionary.

   Returns a list of phones.

   word - lowercase word, no spaces or punctuation
  """
  try:
    return pronouncing.phones_for_word(word)[0].split()
  except IndexError:
    return pin.soundout(word)

def arpabetPlusFromText(string):
  """Turns text into a list of arpabet phones, + STOP for punctuation.
  One stop for ,; two stops for .:?! or double dash.
  Turns a hyphen between letters (as in "use-case") into a space,
   but in "a phrase - like this one" replaces it with a STOP

  Discards all other punctuation and non-alpha characters (including numbers).

  string -- text to convert to phones"""

  lidx = ridx = 0
  
  phones = []
  doublestops = ".:?!"
  singlestops = ",;"

  while (lidx < len(string)):
    if string[lidx].isalpha():
      while ((ridx < len(string)) and (string[ridx].isalpha())):
        ridx += 1
      phones += phones_for_word_fb(string[lidx:ridx].lower())
      lidx = ridx
    else:
      while ((ridx < len(string)) and (not string[ridx].isalpha())):
        ridx += 1
      doublestop = False
      for c in doublestops:
        if c in string[lidx:ridx]:
          doublestop = True
      if doublestop:
        phones += ['STOP0','STOP0']
      elif '--' in string[lidx:ridx]:
        phones += ['STOP0','STOP0']
      elif '- ' in string[lidx:ridx]:
        phones += ['STOP0']
      else:
        singlestop = False
        for c in singlestops:
          if c in string[lidx:ridx]:
            singlestop = True
        if singlestop:
          phones += ['STOP0']
      lidx = ridx

  return phones

class Syllable:
  # initialization takes a list of phones
  # assuming exactly one vowel/stop in them.
  #
  # vowel - vowel phone (w/o stress indicator) or STOP
  # stress - 0 for STOP, 1 for unstressed, 2 2ndary stress, 3 primary stress
  # (note this is different from the arpabet numbers)
  # cfeat - Counter containing a count of the consonant features
  # vfeat - Counter containing a count of the vowel features
  def __init__(self, phoneslist):
    phones = " ".join(phoneslist)
    for v in vowels:
      loc = phones.find(v)
      if (loc != -1):
        self.vowel = v
        if (v == 'STOP'):
          self.stress = 0
        elif (phones[loc + 2] == '0'):
          self.stress = 1
        elif (phones[loc + 2] == '1'):
          self.stress = 3
        elif (phones[loc + 2] == '2'):
          self.stress = 2
      
      self.cfeat = Counter()
      self.vfeat = Counter()
      for p in phoneslist:
        if isVowel(p):
          for feature in pfm[p[0:-1]]:
            self.vfeat[feature] += 1
        else:
          for feature in pfm[p]:
            self.cfeat[feature] += 1
                
def distance(syl1 : Syllable, syl2 : Syllable):
  """A distance metric between two Syllable objects.
  Adjusted by the constant weights in the first cell,
  applied to Manhattan distance on features.
  (STOP is close to every other vowel in features,
  though not in stress)
  """
  cdist = 0
  vdist = 0
  
  sdist = abs(syl1.stress - syl2.stress) * kDistanceWeightStress

  cdiff = difference(syl1.cfeat,syl2.cfeat)
  for f in cdiff:
    cdist += abs(cdiff[f]) * kDistanceWeightConsonants
  
  if (syl1.vowel != 'STOP' and syl2.vowel != 'STOP'):
    vdiff = difference(syl1.vfeat,syl2.vfeat)
    for f in vdiff:
      vdist += abs(vdiff[f]) * kDistanceWeightVowels

  return sdist + vdist + cdist

def difference(ctr1 : Counter, ctr2 : Counter):
  """A new counter that's the difference between two Counters.
  (Counter.subtract mutates the Counter, which isn't what we want.)
  """
  diff = Counter(ctr1)
  for c in ctr2:
    diff[c] -= ctr2[c]
  
  return diff

def isVowel(phone):
  return phone[0:-1] in vowels
  
def syllabifier(phoneslist):
  """Turns a list of phones into a list of Syllable objects."""

  i = j = nextSylStart = 0
  syllableList = []

  while i < len(phoneslist):
    thisSyllablePhones = []
    j = i
    needVowel = True
    
    while j < len(phoneslist) and (needVowel or not isVowel(phoneslist[j])):
      if isVowel(phoneslist[j]):
        needVowel = False
        nextSylStart = j + 1
      thisSyllablePhones.append(phoneslist[j])
      
      j = j + 1
    
    # only add a syllable if we picked up a vowel
    if not needVowel:
      syllableList.append(Syllable(thisSyllablePhones))
      i = nextSylStart
    else:
      i = j

  return syllableList

def canonFit(list1, list2, offset):
  """Given two syllable lists and a canon offset,
  returns the total syllable distance. Lower is rhymier.
  list2 starts offset syllables later."""

  i = totalDist = 0
  while (i + offset < len(list1) and i < len(list2)):
    totalDist += distance(list2[i], list1[i + offset])
    i = i + 1
  
  return totalDist

In [366]:
def generateCanon(alts = 100, tokensPerIncrement = 5, length = 100,
                  prompt = "If only I could be a little more",
                  temperature = 1.2, top_p = 0.9, offset = 10,
                  startingTokens=10):
  """
  alts - number of samples of text to generate to test for rhymes
  tokensPerIncrement - size of each incremental sample
  length - total tokens (including the prompt)
  prompt - start of canon
  temperature, top_p - passed along to GPT-2
  offset - how many syllables the 2nd part of the canon is delayed
  startingTokens - tokens for first generation. Should be more tokens than the
    prompt (tricky, because it's not clear how many tokens the prompt will contain)
  """
  print(prompt)
  canonSoFar = prompt
  seed = 0

  

  for i in range(startingTokens, length, tokensPerIncrement):
    canonSoFarSyllables = syllabifier(arpabetPlusFromText(canonSoFar))  
    bestTrial = canonSoFar
    bestDist = float("inf")

    for j in range(alts):
      seed += 1
      random.seed(seed)
      trial = ai.generate_one(
              prompt=canonSoFar,
              max_length=i,
              temperature=temperature,
              top_p=top_p)
      trialDiff = trial[len(canonSoFar):]
      print("next text: " + trialDiff)
      trialSyllables = canonSoFarSyllables + syllabifier(arpabetPlusFromText(trialDiff))
      if len(trialSyllables) > len(canonSoFarSyllables): # only test generations with real new text
        trialFit = canonFit(trialSyllables, canonSoFarSyllables, offset)
        if trialFit < bestDist:
          bestDist = trialFit
          bestTrial = trial
    
    canonSoFar = bestTrial
    print(canonSoFar)

  print()
  print("FINAL CANON:")
  print(canonSoFar)









In [367]:
generateCanon(alts = 50, tokensPerIncrement=2, length=200, prompt="If only I could be", offset = 4, temperature = 1, top_p = 0.9)


If only I could be
next text:  like her.
That
next text:  like this.
Start
next text:  beautiful.
Doing
next text:  beautiful.
Let's
next text:  your father, a mom
next text:  so beautiful.
You
next text:  saved, but it would
next text:  saved!
I don
next text:  your father.
Oh
next text:  this beautiful.
I
next text:  my real self today.
next text:  like him.
I
next text:  my best self, so
next text:  the one to watch the
next text:  as beautiful as this.
next text:  like her.
Like
next text:  as beautiful as this.
next text:  the one to ask for
next text:  your father, too.
next text:  allowed to have that conversation
next text:  saved.
I can
next text:  my best self and not
next text:  in that boat.

next text:  saved.
I'm
next text:  so beautiful.
And
next text:  saved.
I'd
next text:  like her.
Like
next text:  like him.
Like
next text:  your father today.

next text:  beautiful.
And I
next text:  yourself, a little closer
next text:  my best self and not
next text:  as beautiful

KeyboardInterrupt: ignored



```
If  only I  could be in that  boat.       That boat.
            If    only  I     could be in that boat.

I    don't know why.       Cousinwog, cousin of  Brassy and
That boat.         I don't know why.  Cousinwog,     cousin 

Red from    Cousinwog  and   Mighty from   Cousinwog  
of  Brassy     and Red from  Cousinwog    and Mighty from

City  and  Plateaus  aren't  together        anymore.   They're
Cousinwog  City      and     Plateaus aren't together anymore. 

having a breakout and Plateaus says "What's going    on    here?"
      They're having   a breakout    and    Plateaus says "What's

going on here?"
```

(offset of 4, temperature .7)




```
If only I  could be my good   editor     in one piece 
        If only     I  could  be my good editor

-- which, at least,   it   was, when  absolutely forced and wri-
editor   --  which at least,    it   was, when absolutely

ting   ridiculously    long chunks of crap chunks.
forced and writing ridiculously       long chunks of crap

Without the   ethics.          "It sucks."    Sometimes,
chunks.       Without  the  ethics.       "It sucks."

sometimes, it's just "it's difficult       to tell a
Sometimes, sometimes,      it's just "it's difficult

story"   as art,   or an illusion.    There's another,     ever
to tell  a  story" as art, or an illusion.       There's another,

another trance.  Sometimes it's "I don't  think it's
   ever another  trance.        Sometimes it's  "I

important"         as   entertainment.
don't think it's   important" as  entertainment.
```
(offset of 3, temperature 2.0, vowels 4, stress 3, consonants 5)


```
If only I could be allowed to  be loved.     Been  avoiding
          If    only  I  could be allowed to be    loved.

eye  doctor since  ninety-two.
Been avoiding      eye doctor  since ninety-two.
```

(offset of 4, temperature 2, vowels, stress, cons = 2,1,2; 3 tokens per increment)

```
If only I could be that good.
People say it - "It's true it isn't" - "It is", "It is true it is true" - "Dear @lukejchengnott, in what way?"
```

(same settings, but offset of 3)

```
If only
I could be
as good as
this.
I know I'm
doing ev
ery thing right.
          I
know I'm do
ing every
thing I can.
          I
know I'm do
ing every
thing I can.
          I
know I'm cap
able.
     I be
lieve I am
loved.
I believe
I am ent
ertaining
the impos
sible.
  But all
of a   sud
den it seems
like it's a
bout to col
lapse on it
self and I
have to find
a place to
stand,  so
I don't lis
ten.
Finally
I find a
place to sit,
   thinking
it's okay
to be there.
      There
is a rea
sonable
place to go,
    but I'm
worried it
will be rai
ded by the
big demon
ic fuckups,
   and killed.
          I
know that I
am allowed
to feel my
anger and
anxie
ty under
control,
that this is
an extreme
ly danger
ous place.
    I sit
on it,
thinking it's
really o
kay to be
there.
But then I
get really
worried and
angry   and
a     pile  of
Trash piles up.
```

(alts -> 50 from 250, 5 tokens per increment, temperature 1)

(stress weight 1->200, offset -> 3)

In [363]:
kDistanceWeightVowels = 2.0
kDistanceWeightStress = 5.0
kDistanceWeightConsonants = 5.0

s = syllabifier(arpabetPlusFromText("the road the road the road the road"))
canonFit(s,s,2)

30.0

todo: don't choke on accented characters.